# IA - Projeto 01: Horário de turma 

# 1. Introdução
### Membros do grupo
- Hugo Ferreira Baptista — nº 23279
- Nuno da Cunha Faria Gajo — nº 23002


### Contexto e Objetivo do projeto
O objetivo deste projeto é para desenvolver um **agente inteligente** capaz de gerar **horários de turmas** que respeitem certas restrições tais como, disponibilidade de professores, salas e evitar conflitos de horários.  
O problema é formulado como um **CSP** utilizando Python e a biblioteca `python-constraint`.


In [ ]:
# Install contraint library
%pip install python-constraint

In [ ]:
# Import contraint library
from constraint import *
from itertools import combinations
from collections import defaultdict

# 2. Design do Agente

Nesta seção, definimos o problema do horário de turmas como um **CSP**.

Cada variável representa uma **aula** e pode assumir um valor (bloco de tempo, sala, online) onde:
- `bloco de tempo` \ `block` ∈ [1; 20] (20 blocos por semana);
- `sala` \ `room` ∈ {R1, R2, R3, Lab01};
- `online` ∈ {Verdadeiro, Falso}.

Nós definimos:
- Variáveis e domínios;  
- Restrições rígidas (obrigatórias);
- Restrições flexíveis (opcionais, tratadas posteriormente);
- Heurísticas para melhorar o desempenho;

In [ ]:
# Exemplo de leitura do ficheiro e preparação dos dados:
with open('ClassTT_01_tiny.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        print(line.strip())
        

In [ ]:
# =====================================================
#   TIMETABLE CSP - SOLUÇÃO FINAL (AJUSTADA)
# =====================================================

# --- Recursos ---
rooms = ['R1', 'R2', 'R3', 'Lab01']

classes = {
    't01': ['UC11','UC12','UC13','UC14','UC15'],
    't02': ['UC21','UC22','UC23','UC24','UC25'],
    't03': ['UC31','UC32','UC33','UC34','UC35']
}

teachers = {
    'jo': ['UC11','UC21','UC22','UC31'],
    'mike': ['UC12','UC23','UC32'],
    'rob': ['UC13','UC14','UC24','UC33'],
    'sue': ['UC15','UC25','UC34','UC35']
}

teacher_restrictions = {
    'mike': [13,14,15,16,17,18,19,20],
    'rob': [1,2,3,4],
    'sue': [9,10,11,12,17,18,19,20]
}

fixed_rooms = {'UC14': 'Lab01', 'UC22': 'Lab01'}

# Apenas estas duas UCs podem ser online
online_allowed = ['UC21', 'UC31']

# --- Inicializar problema ---
problem = Problem()

# Variáveis: (slot, sala, online)
all_vars = []
for turma, ucs in classes.items():
    for uc in ucs:
        for i in [1,2]:
            var = f"{uc}_{i}"
            all_vars.append(var)
            
            # professor
            teacher = next(t for t,courses in teachers.items() if uc in courses)
            unavailable = teacher_restrictions.get(teacher, [])
            
            # domínio inicial
            if uc in online_allowed:
                domain = [(slot, room, online)
                          for slot in range(1,21) if slot not in unavailable
                          for room in rooms
                          for online in [True, False]]
            else:
                domain = [(slot, room, False)
                          for slot in range(1,21) if slot not in unavailable
                          for room in rooms]
            
            # sala fixa
            if uc in fixed_rooms:
                domain = [(s,r,o) for (s,r,o) in domain if r == fixed_rooms[uc]]
            
            problem.addVariable(var, domain)
# =====================================================
#                  HARD CONSTRAINTS
# =====================================================

# --- 1. No máximo 3 aulas por dia por turma ---
def max3_por_dia(*vals):
    dias = defaultdict(int)
    for v in vals:
        dia = (v[0]-1)//4
        dias[dia] += 1
        if dias[dia] > 3:
            return False
    return True

for turma, ucs in classes.items():
    vars_turma = [f"{uc}_{i}" for uc in ucs for i in [1,2]]
    problem.addConstraint(max3_por_dia, vars_turma)

# --- 3. Nenhuma sala usada ao mesmo tempo ---
def no_room_conflict(*vals):
    used = set()
    for v in vals:
        key = (v[0], v[1])  # slot, room
        if key in used:
            return False
        used.add(key)
    return True


for v1, v2 in combinations(all_vars,2):
    problem.addConstraint(lambda a,b: a[0]!=b[0] or a[1]!=b[1], (v1,v2))


# --- 4. Nenhuma turma com 2 aulas ao mesmo tempo ---
for turma, ucs in classes.items():
    vars_turma = [f"{uc}_{i}" for uc in ucs for i in [1,2]]
    for v1, v2 in combinations(vars_turma,2):
        problem.addConstraint(lambda a,b: a[0]!=b[0], (v1,v2))



# --- 8. Cada turma tem pelo menos uma aula na segunda (dia 1) ---
def at_least_one_day1(*vals):
    return any((v[0]-1)//4 == 0 for v in vals)

for turma, ucs in classes.items():
    vars_turma = [f"{uc}_{i}" for uc in ucs for i in [1,2]]
    problem.addConstraint(at_least_one_day1, vars_turma)

# --- 9. Professores não podem dar aulas ao mesmo tempo ---

for prof, ucs_prof in teachers.items():
    vars_prof = [f"{uc}_{i}" for uc in ucs_prof for i in [1,2]]
    for v1, v2 in combinations(vars_prof,2):
        problem.addConstraint(lambda a,b: a[0]!=b[0], (v1,v2))

# =====================================================
#                  SOLUÇÃO
# =====================================================

print("⏳ Gerando solução...")
solution = problem.getSolution()

if not solution:
    print("❌ Nenhuma solução encontrada.")
else:
    print("\n✅ Solução encontrada!\n")

    horas = ["09-11", "11-13", "14-16", "16-18"]

    # Organizar resultados por turma
    for turma, ucs in classes.items():
        print(f"\n==============================")
        print(f"📅 Horário da Turma {turma.upper()}")
        print(f"==============================")
        
        tabela = {dia: {h: "" for h in horas} for dia in range(1,6)}

        for uc in ucs:
            for i in [1,2]:
                var = f"{uc}_{i}"
                slot, room, online = solution[var]
                dia = (slot-1)//4 + 1
                hora = horas[(slot-1)%4]
                modo = "ON" if online else "PR"
                tabela[dia][hora] = f"{uc} ({room}, {modo})"

        print(f"{'Hora':<10}  Dia1              Dia2              Dia3              Dia4              Dia5")
        print("-"*95)
        for h in horas:
            linha = f"{h:<10}  "
            for dia in range(1,6):
                cel = tabela[dia][h] if tabela[dia][h] else "-"
                linha += f"{cel:<18} "
            print(linha)
        print("\n")


In [36]:
# === Dados ===
classes = {
    "t01": ["UC11","UC12","UC13","UC14","UC15"],
    "t02": ["UC21","UC22","UC23","UC24","UC25"],
    "t03": ["UC31","UC32","UC33","UC34","UC35"]
}

teachers = {
    "jo": ["UC11","UC21","UC22","UC31"],
    "mike": ["UC12","UC23","UC32"],
    "rob": ["UC13","UC14","UC24","UC33"],
    "sue": ["UC15","UC25","UC34","UC35"]
}

teacher_restrictions = {
    "mike": [13,14,15,16,17,18,19,20],
    "rob": [1,2,3,4],
    "sue": [9,10,11,12,17,18,19,20]
}

fixed_rooms = {
    "UC14":"Lab01",
    "UC22":"Lab01"
}

online_allowed = {"UC21", "UC31"}

rooms = ["R1","R2","R3","Lab01"]

slots_per_day = 4
total_days = 5
total_slots = slots_per_day * total_days

# === Criar domínios de cada aula ===
all_vars = {}
for turma, ucs in classes.items():
    for uc in ucs:
        for i in [1,2]:
            var = f"{uc}_{i}"
            teacher = next(t for t,courses in teachers.items() if uc in courses)
            unavailable = teacher_restrictions.get(teacher, [])

            domain = []
            for slot in range(1, total_slots + 1):
                if slot in unavailable:
                    continue
                for room in rooms:
                    if uc in fixed_rooms and room != fixed_rooms[uc]:
                        continue
                    for online in ([True, False] if uc in online_allowed else [False]):
                        domain.append((slot, room, online))
            all_vars[var] = domain

# === Backtracking turma a turma ===
solution = {}

def is_valid_assignment(var, val, current_solution):
    # Professores
    teacher = next(t for t,courses in teachers.items() if var[:4] in courses)
    for v, assigned in current_solution.items():
        t2 = next(t for t,courses in teachers.items() if v[:4] in courses)
        if teacher == t2 and val[0] == assigned[0]:
            return False
    # Turmas
    turma_var = next(t for t,courses in classes.items() if var[:4] in courses)
    for v, assigned in current_solution.items():
        turma_v = next(t for t,courses in classes.items() if v[:4] in courses)
        if turma_var == turma_v and val[0] == assigned[0]:
            return False
    # Salas
    for assigned in current_solution.values():
        if val[0] == assigned[0] and val[1] == assigned[1]:
            return False
    # Máximo 3 aulas/dia por turma
    dia = (val[0]-1)//slots_per_day + 1
    count_dia = 0
    for v, assigned in current_solution.items():
        if (assigned[0]-1)//slots_per_day + 1 == dia:
            turma_v = next(t for t,courses in classes.items() if v[:4] in courses)
            if turma_v == turma_var:
                count_dia += 1
    if count_dia >= 3:
        return False
    return True

def backtrack_turma(vars_turma, current_solution):
    if not vars_turma:
        return True
    var = min(vars_turma, key=lambda v: len(vars_turma[v]))  # menor domínio primeiro
    domain = vars_turma[var]
    for val in domain:
        if is_valid_assignment(var, val, current_solution):
            current_solution[var] = val
            new_vars = vars_turma.copy()
            new_vars.pop(var)
            if backtrack_turma(new_vars, current_solution):
                return True
            current_solution.pop(var)
    return False

# --- Resolver turma a turma ---
for turma, ucs in classes.items():
    vars_turma = {}
    for uc in ucs:
        for i in [1,2]:
            var = f"{uc}_{i}"
            vars_turma[var] = all_vars[var]
    if not backtrack_turma(vars_turma, solution):
        print(f"❌ Nenhuma solução encontrada para a turma {turma}")
        break

if not solution:
    print("❌ Nenhuma solução encontrada.")
else:
    print("\n✅ Solução encontrada!\n")
    horas = ["09-11", "11-13", "14-16", "16-18"]

    for turma, ucs in classes.items():
        print(f"\n==============================")
        print(f"📅 Horário da Turma {turma.upper()}")
        print(f"==============================")

        tabela = {dia: {h: "" for h in horas} for dia in range(1,6)}

        for uc in ucs:
            for i in [1,2]:
                var = f"{uc}_{i}"
                slot, room, online = solution[var]
                dia = (slot-1)//4 + 1
                hora = horas[(slot-1)%4]
                modo = "ON" if online else "PR"
                tabela[dia][hora] = f"{uc} ({room}, {modo})"

        print(f"{'Hora':<10}  Dia1              Dia2              Dia3              Dia4              Dia5")
        print("-"*95)
        for h in horas:
            linha = f"{h:<10}  "
            for dia in range(1,6):
                cel = tabela[dia][h] if tabela[dia][h] else "-"
                linha += f"{cel:<18} "
            print(linha)
        print("\n")



✅ Solução encontrada!


📅 Horário da Turma T01
Hora        Dia1              Dia2              Dia3              Dia4              Dia5
-----------------------------------------------------------------------------------------------
09-11       UC12 (R1, PR)      UC14 (Lab01, PR)   UC13 (R1, PR)      UC11 (R1, PR)      -                  
11-13       UC12 (R1, PR)      UC14 (Lab01, PR)   UC13 (R1, PR)      -                  -                  
14-16       UC15 (R1, PR)      UC15 (R1, PR)      UC11 (R1, PR)      -                  -                  
16-18       -                  -                  -                  -                  -                  



📅 Horário da Turma T02
Hora        Dia1              Dia2              Dia3              Dia4              Dia5
-----------------------------------------------------------------------------------------------
09-11       UC22 (Lab01, PR)   UC23 (R1, PR)      UC21 (R2, ON)      -                  -                  
11-13       UC22

# 3. Agente em execução
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nunc pulvinar semper augue vel porttitor. Aenean porta nulla congue sem venenatis, non varius lorem imperdiet. Aenean odio lacus, fringilla a libero ut, ullamcorper finibus lectus. Nunc ullamcorper eu urna volutpat egestas. Nunc vehicula maximus quam non lobortis. Ut eget dolor quis arcu vestibulum consectetur. Aliquam consequat lectus odio, in scelerisque orci lacinia sit amet.Lorem ipsum dolor sit amet, consectetur adipiscing elit. Curabitur vitae sapien urna. Ut at aliquet metus, vel porta metus. Curabitur quis tellus at sapien ullamcorper feugiat. Donec auctor, ante at ornare condimentum, magna mi ornare orci, vitae feugiat ligula sapien sed ex. Cras id interdum ipsum. Sed metus nibh, maximus at viverra at, pretium id est. Vivamus semper urna et libero gravida, euismod tincidunt sem hendrerit. Curabitur semper magna eu nisl laoreet congue. Curabitur viverra odio sed neque volutpat, volutpat feugiat nunc finibus. Donec mattis semper odio, eget tincidunt nulla convallis ut. Donec odio diam, facilisis vitae iaculis quis, gravida vel lacus. Sed finibus arcu sit amet dolor lobortis lacinia. Proin vestibulum lectus non lorem tempus aliquam. Cras ex nulla, condimentum sed mauris at, pulvinar finibus erat.

# 4. Conclusão
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nunc pulvinar semper augue vel porttitor. Aenean porta nulla congue sem venenatis, non varius lorem imperdiet. Aenean odio lacus, fringilla a libero ut, ullamcorper finibus lectus. Nunc ullamcorper eu urna volutpat egestas. Nunc vehicula maximus quam non lobortis. Ut eget dolor quis arcu vestibulum consectetur. Aliquam consequat lectus odio, in scelerisque orci lacinia sit amet.